## 한국음식 분류기

In [1]:
## 사용할 모델 라이브러리 import
import sys, os
#from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Activation
from keras.layers import Dropout
from keras.layers import Flatten
#from keras.layers import Dense
#from keras.utils import np_utils
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical  
import pickle
import matplotlib.pyplot as plt

In [2]:
## 초기 설정
root_dir = "../data/"
categories = ["fire", "smoke"]
nb_classes = len(categories)
image_size = 256

In [3]:
# 데이터 로딩  (1)
def load_dataset():
    with open(root_dir + "fire_surveillance.pkl", "rb") as f:
        x_train, x_test, y_train, y_test = pickle.load(f)
   # x_train, x_test, y_train, y_test = np.load("./kfood/kfood.npy", allow_pickle=True)
    x_train = x_train.astype("float32") / 256
    x_test = x_test.astype("float32") / 256
    y_train = to_categorical(y_train, nb_classes)
    y_test = to_categorical(y_test, nb_classes)
    return  x_train, x_test, y_train, y_test

In [4]:
# 모델 구성  (2)
# padding='same' : 출력 이미지 사이즈가 입력 이미지 사이즈와 동일
# ‘valid’인 경우 출력 이미지 크기가 입력 이미지 크기보다 작아짐
# 참조 : https://tykimos.github.io/2017/01/27/CNN_Layer_Talk/
def build_model(in_shape):
    model = Sequential()

    model.add(Conv2D(32, (3, 3), padding='same', input_shape=in_shape))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))  # 수정됨
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding='same'))    
    model.add(Activation('relu'))
    
    model.add(Conv2D(64, (3, 3)))  # padding 기본값: 'valid'
    model.add(MaxPooling2D(pool_size=(2, 2)))  # 수정됨
    model.add(Dropout(0.25))

    # model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))

    model.add(Flatten())
    model.add(Dense(nb_classes))  # 클래스 수만큼
    model.add(Activation('softmax'))

    model.compile(
        loss='categorical_crossentropy',  # 수정됨
        optimizer='rmsprop',
        metrics=['accuracy']
    )

    return model

In [5]:
# 모델 학습을 수행하고 저장된 모델을 파일로 저장  (3)
def model_train(x, y):
    model = build_model(x.shape[1:])
    global history
    
    history = model.fit(x, y, batch_size=32, epochs=10)

    return model 

In [6]:
# 모델 평가하기  (4)
def model_eval(model, x, y):
    score = model.evaluate(x, y)
    print('loss=', score[0])
    print('accuracy=', score[1])

In [7]:
# 모델 학습 및 평가
x_train, x_test, y_train, y_test = load_dataset()

In [8]:
x_train.shape

(1719, 256, 256, 3)

In [9]:
y_train[0]

array([0., 1.])

In [10]:
y_train.shape

(1719, 2)

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense

model = model_train(x_train, y_train)
model_eval(model, x_test, y_test)

C:\anaconda\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 50s 910ms/step - accuracy: 0.8194 - loss: 7.3996
Epoch 2/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 49s 903ms/step - accuracy: 0.9647 - loss: 0.3880
Epoch 3/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 50s 922ms/step - accuracy: 0.9924 - loss: 0.0400
Epoch 4/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 52s 959ms/step - accuracy: 0.9937 - loss: 0.0421
Epoch 5/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 51s 946ms/step - accuracy: 0.9932 - loss: 0.0530
Epoch 6/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 52s 961ms/step - accuracy: 0.9839 - loss: 0.6312
Epoch 7/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 50s 930ms/step - accuracy: 0.9966 - loss: 0.0136
Epoch 8/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 50s 926ms/step - accuracy: 0.9977 - loss: 0.0208
Epoch 9/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 50s 932ms/step - accuracy: 0.9743 - loss: 0.1473
Epoch 10/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 51s 940ms/step - accuracy: 0.9954 - loss: 0.0157
18/18 ━━━━━━━━━━━━━━━━━━━━ 4s 212ms/step - accuracy: 0.9898 - loss: 0.0385
loss= 0.02667621336877346
accuracy= 0.99303138256073


In [12]:
# 모델 저장
model.save("../data/fire_surveillance_model.h5")

In [13]:
# plt.figure(figsize=(12, 5))

# # 손실 그래프
# plt.subplot(1, 2, 1)
# plt.plot(history.history["loss"], label="Train Loss")
# plt.plot(history.history["val_loss"], label="Validation Loss")
# plt.title("Model Loss")
# plt.xlabel("Epoch")
# plt.ylabel("Loss")
# plt.legend()

# plt.subplot(1, 2, 2)
# plt.plot(history.history["accuracy"], label="Train Accuracy")
# plt.plot(history.history["val_accuracy"], label="Validation Accuracy")
# plt.title("Model Accuracy")
# plt.xlabel("Epoch")
# plt.ylabel("Accuracy")
# plt.legend()

# plt.tight_layout()
# plt.show()